## 1 Exercise: Generate random data
### The idea is to learn how to generate random data following a gaussian distribution with different variances. Moreover we will generate a specific outlier in order to study how it affects a linear regression assuming different metrics.

In [1]:
# 1) Generate an array array (with length=20) of standard deviations of random values following a uniform distribution 
# between 2 and 3. Fix the value of the last element = 5. The last point will be the outlier.

import numpy as np


std_array = np.random.uniform(2, 3, 20)
std_array[-1] = 5
print(std_array)

[2.1103318  2.36630944 2.73926903 2.3947315  2.00345466 2.17202112
 2.24406792 2.88140898 2.08467061 2.92904147 2.21621326 2.54711488
 2.10871895 2.69330392 2.65911735 2.45406263 2.55300175 2.85110093
 2.00036576 5.        ]
